In [ ]:
# On importe toutes les librairies utiles 
import seaborn as sns
import pandas as pd
import string
from nltk.tokenize import word_tokenize
import nltk
from nltk.stem import WordNetLemmatizer   

In [ ]:
# On charge les données dans sms 
sms = pd.read_csv("/kaggle/input/sms-spam-collection-dataset/spam.csv", encoding='latin-1')
sms.head() 

In [ ]:
#on supprime les 3 autres colonnes avec drop 
sms=sms.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4'] ) 
sms 

In [ ]:
#on renomme les deux colonnes v1 et V2
sms=sms.rename(columns={'v1':'label','v2':'message'})
sms

In [ ]:
#on affiche le diagramme de label avec countplot 
ax=sns.countplot(x="label", data=sms) 

In [ ]:
#On affiche les dimensions de la dataframe 
sms.shape

# Questions 9: nettoyage

In [ ]:
#mettre en minuscule 
sms['message']=sms['message'].str.lower()
sms       

In [ ]:
#On affiche les punctuations pour voir ce qu'on va enelever  
string.punctuation

In [ ]:
# On créé la fonction remove punct 
def remove_punct(text):
    text_tok = word_tokenize(text)
    A=[]
    for word in text_tok: 
        if not word in string.punctuation:
            A.append(word)
            
    resultat=" ".join(A)  
    return resultat

In [ ]:
# On applique la fonction de retrait des ponctuations 
sms.message=sms.message.apply(remove_punct)
sms

In [ ]:
#Télécharger les stopwords
from nltk.corpus import stopwords
# on affiche les stopwords en les incrémentant dans stop
stop=set(stopwords.words('english'))
stop

In [ ]:
# on créé la fonction de retrait des stopsword
def remove_stopword(text):
    text_tok = word_tokenize(text)
    A = []
    for a in text_tok:
        if not a in stop:
            A.append(a)
            
    resultat =" ".join(A)
    return resultat

In [ ]:
# on applique la fonction de retrait des stopword et on affiche le résultat
sms.message=sms.message.apply(remove_stopword)
sms

In [ ]:

lemmatizer=WordNetLemmatizer()

In [ ]:
#On créé la fonction de lemmetizing
def lemm(text):
    text_tok = word_tokenize(text) 
    l=[]
    for word in text_tok:
        l.append(lemmatizer.lemmatize(word))
        
    resultat = " ".join(l)

    return resultat


In [ ]:
#On applique la fonction de lemmetizing et on affiche le résultat
sms.message=sms.message.apply(lemm)
sms

# la colonne message est nettoyée

In [ ]:
#On affecte les messages dans corpus afin de s'en reservir 
corpus=sms['message'].values
corpus

# Bag of words


In [ ]:
#On vectorise pour bag of words 
from sklearn.feature_extraction.text import CountVectorizer
bw_vect = CountVectorizer()
bw_fit=bw_vect.fit(corpus)
# on vectorise les mots 
bw_corpus = bw_fit.transform(corpus)

In [ ]:
#on affiche sous forme de dataframe 
bw_sms=pd.DataFrame(bw_corpus.toarray(),columns=bw_fit.get_feature_names())
bw_sms

In [ ]:
# On créé et sauvegarde dans XX et Y 
from sklearn.model_selection import train_test_split
Y=sms.label
XX=bw_sms
# On divise les deux variables en deux parties chacunes  
XX_train, XX_test, Y_train, Y_test = train_test_split(XX, Y, test_size=0.3, random_state=0)

In [ ]:
# On paramètre l'arbre de décision 
from sklearn import tree
tree_model = tree.DecisionTreeClassifier(max_depth = 2)
tree_model=tree_model.fit(XX_train, Y_train)

In [ ]:
#On compte les valeurs dans Y_train
Y_train.value_counts()

In [ ]:
# On affiche l'arbre de décision en paramerant sa taille et ses titres 
import matplotlib.pyplot as plt
plt.figure(figsize=(15,10))
names = ['spam', 'ham']
tree.plot_tree(tree_model,feature_names = XX.columns, 
               class_names=names,
               filled = True)

In [ ]:
#On regarde si XX test contient des spam et on sauvegarde dans Y_Predict 
Y_predict=tree_model.predict(XX_test)

In [ ]:
# On affiche la matrice de confusion entre Y_Predict et Y_test 
from sklearn.metrics import accuracy_score, confusion_matrix 
mat = confusion_matrix(Y_predict,Y_test)
print(mat)

In [ ]:
# On affiche sous forme graphique la matrice de confusion avec une heatmap
sns.heatmap(mat, annot=True,  xticklabels=names, yticklabels=names)
plt.xlabel('Test')
plt.ylabel('Predicted')

On a énormément de spam, on remarque que on a des erreur de tri entre les predictions et le test (41 confusions entre prédit ham et testé spam, 92 prédit spam et testés ham). 

# TFI DF

In [ ]:
#On importe la fonction 
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
#On initialise les paramètres du vectoriseur 
tf_vect = TfidfVectorizer(max_features=5000)  
tfidf_fit=tf_vect.fit(corpus)
#on vectorise le corpus déclaré plus haut 
tfidf_corpus= tfidf_fit.transform(corpus)


In [ ]:
# on affiche les vecteurs 
tfidf_fit.get_feature_names()

In [ ]:
#on met sous forme de dataframe 
tfidf_data=pd.DataFrame(tfidf_corpus.toarray(),columns=tfidf_fit.get_feature_names())
tfidf_data

In [ ]:
#on enregistre dans X et Y label et le corpus 
from sklearn.model_selection import train_test_split
Y=sms.label
X=tfidf_corpus 
# on utilise la fonction train test split 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

In [ ]:
#on implémente les méthodes fit et predict
from sklearn import tree
#on paramètre l'arbre de décision 
tree_model = tree.DecisionTreeClassifier(max_depth = 2)
tree_model=tree_model.fit(X_train, Y_train)

In [ ]:
#on compte les valeurs contenue dans le Y_train 
Y_train.value_counts()

In [ ]:
#on affiche l'arbre de décision en paramétrant son affichage 
import matplotlib.pyplot as plt
plt.figure(figsize=(15,10))
names = ['spam', 'ham']
tree.plot_tree(tree_model,feature_names = Y_train, 
               class_names=names,
               filled = True)

In [ ]:
#on stocke dans Y predict la réponse 
Y_predict=tree_model.predict(X_test)

In [ ]:
# on calcule la matrice de confusion 
from sklearn.metrics import accuracy_score, confusion_matrix 
mat = confusion_matrix(Y_predict,Y_test)
print(mat)

In [ ]:
#On  affiche la matrice confusion sous forme de heatmap
sns.heatmap(mat, annot=True,  xticklabels=names, yticklabels=names)
plt.xlabel('Test')
plt.ylabel('Predicted')

**Question 16**
On peut voir que l'on a 11 confusions entre des messages prédis comme ham et testé comme spam, on a également 118 confusions entre la prediction de spam et le test qui les a classé comme ham. 

In [ ]:
#on importe gridsearch 
from sklearn.model_selection import GridSearchCV
import numpy as np

In [ ]:
# On déclare toutes les deths que l'on veut tester 
depths = np.arange(10, 40,5) # The maximum depth of the tree
param_grid = [{'max_depth':depths}]
param_grid

In [ ]:
#on reprend l'arbre avec les différents deths 
grid_tree= GridSearchCV(estimator=tree.DecisionTreeClassifier(),param_grid=param_grid,scoring='accuracy',cv=10)
grid_tree.fit(X_train, Y_train)
best_model_tree = grid_tree.best_estimator_

In [ ]:
#On affiche le résultat 
grid_tree.best_params_

# Question 18

In [ ]:
# on importe les methodes des foret d'arbres 
from sklearn.ensemble import RandomForestClassifier

In [ ]:
Rf_model = RandomForestClassifier()
Rf_model=Rf_model.fit(X_train, Y_train)


In [ ]:
#on calcule le accuracy score 
Y_predict=Rf_model.predict(X_test)
a_CART = accuracy_score(Y_test,Y_predict)
print("Accuracy score: ",a_CART)

In [ ]:
#on affiche la matrice de confusion
mat = confusion_matrix(Y_predict, Y_test)
sns.heatmap(mat, annot=True,  xticklabels=names, yticklabels=names)
plt.xlabel('Test')
plt.ylabel('Predicted')

# Question 19

In [ ]:
tfidf_data=pd.DataFrame(tfidf_corpus.toarray(),columns=tfidf_fit.get_feature_names())
tfidf_data

In [ ]:
def classer(text):
    text=text.lower()
    text=remove_punct(text)
    text=remove_stopword(text)
    text=lemm(text)
    tfidf_text=tfidf_fit.transform([text])
    resultat=Rf_model.predict(tfidf_text)
    
    #if text == Y_test spam
    #print ('spam')
    #if text == Y_test spam 
    #print('ham')
    
    
   
    
    return resultat

    

# Question 20

In [ ]:
while True:
    text = str(input("Input: "))
    if text== "exit":
        print("Response: Exiting.....")
        break
    print("Response:",classer(text))
